In [ ]:
import multiprocessing
import os

os.environ['JAX_PLATFORMS'] = 'cuda,cpu'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '1.0'
os.environ["XLA_FLAGS"] = f"--xla_force_host_platform_device_count={os.cpu_count()}"

from tqdm import tqdm
import jax.random
import astropy.units as au
from dsa2000_assets.content_registry import fill_registries
from dsa2000_assets.registries import misc_registry
from dsa2000_common.common.logging import dsa_logger
from dsa2000_fm.bright_souces.evaluate_rms import simulate_rms


def run(result_idx, cpu_idx, gpu_idx, pointing):
    cpus = jax.devices("cpu")
    gpus = jax.devices("cuda")
    cpu = cpus[cpu_idx]
    gpu = gpus[gpu_idx]
    simulate_rms(
        cpu=cpu,
        gpu=gpu,
        result_num=result_idx,
        seed=0,
        save_folder='sky_loss_19mar2025_full_survey',
        array_name='dsa2000_optimal_v1',
        pointing=pointing,
        num_measure_points=256,
        sim_batch_size=128,
        angular_radius=1.75 * au.deg,
        prior_psf_sidelobe_peak=1e-3,
        bright_source_id='nvss_calibrators',
        pointing_offset_stddev=1 * au.arcmin,
        axial_focus_error_stddev=1 * au.mm,
        horizon_peak_astigmatism_stddev=1 * au.mm,
        turbulent=True,
        dawn=True,
        high_sun_spot=True,
        with_ionosphere=True,
        with_dish_effects=True,
        with_smearing=True
    )


def main(node_idx: int, num_nodes: int):
    cpus = jax.devices("cpu")
    gpus = jax.devices("cuda")
    dsa_logger.info(f"Launching over {len(gpus)} gpus")
    queues = [multiprocessing.Queue() for _ in gpus]

    fill_registries()
    survey_pointings = misc_registry.get_instance(misc_registry.get_match('survey_pointings'))
    pointings = survey_pointings.survey_pointings_v1()

    # fill queues with input args
    result_idx = 0
    node_id = 0
    for pointing in tqdm(pointings):
        if (node_id % num_nodes) == node_idx:
            q = queues[result_idx % len(gpus)]
            gpu_idx = result_idx % len(gpus)
            cpu_idx = result_idx % len(cpus)
            q.put((run, result_idx, cpu_idx, gpu_idx, pointing))
        result_idx += 1
        node_id += 1

    # now run the jobs in thread pool
    def worker(queue):
        while True:
            args = queue.get()
            if args is None:
                break
            f = args[0]
            args = args[1:]
            f(*args)

    # now run the jobs in thread pool, each job processes a queue
    import concurrent.futures

    with concurrent.futures.ProcessPoolExecutor() as executor:
        for q in queues:
            executor.submit(worker, q)


main(0, 2)
